# Convolutional model implementtion 
## LeNet 5 
### Using Con2d as a convolutional layer and Linear as a linear layer from torch

In [1]:
import torch
import torch.nn as nn #parent
import torch.nn.functional as F #activation func, max pooling funcs
from torch.utils.data import ConcatDataset
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import tensorflow as tf

import math
import numpy as np

import sys
sys.path.append('/home/sandrech/code/ml/square_attack_v2')
from attack import square_attack_l2, square_attack_linf

import utils
from utils import dense_to_onehot

## Creating model

In [ ]:
class LeNet_5(nn.Module):
    def __init__(self):
        super(LeNet_5, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) #equivalent to nn.Maxpool2d(2)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # x = F.max_pool2d(F.relu(self.conv2(x)), 2)

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features = num_features * s
        return num_features
    def predict(self, x, batch_size=32):

        if x.shape[0] == 0:
            raise ValueError("Входные данные не могут быть пустыми.")
        x = torch.as_tensor(x, dtype=torch.float32)
        self.eval()
        n_batches = math.ceil(x.shape[0] / batch_size)
        logits_list = []
        with torch.no_grad():  
            for i in range(n_batches):
                x_batch = x[i * batch_size:(i + 1) * batch_size]
                #x_batch = x_batch.to(device='cuda')
                logits = self(x_batch).cpu().numpy() 
                logits_list.append(logits)
        logits = np.vstack(logits_list)
        return logits
    def loss(self, y, logits, targeted=False, loss_type='margin_loss'):
        if loss_type == 'margin_loss':
            preds_correct_class = (logits * y).sum(1, keepdims=True)
            diff = preds_correct_class - logits  # difference between the correct class and all other classes
            diff[y] = np.inf  # to exclude zeros coming from f_correct - f_correct
            margin = diff.min(1, keepdims=True)
            loss = margin * -1 if targeted else margin
        elif loss_type == 'cross_entropy':
            probs = utils.softmax(logits)
            loss = -np.log(probs[y])
            loss = loss * -1 if not targeted else loss
        else:
            raise ValueError('Wrong loss.')
        return loss.flatten()

## Downloading datasets from torchvision and applying transforms to it

In [3]:
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)
x = torch.stack([sample[0] for sample in ConcatDataset([trainset])])
mean = torch.mean(x, dim=(0,2,3))
std = torch.std(x, dim=(0,2,3)) #standart deviation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, std)])

Files already downloaded and verified


## Organizing data into batches

In [4]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

## Adding optimizer and loss function

In [5]:
net = LeNet_5()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Training model

In [6]:
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

#st = net.state_dict()
#torch.save(st, "conv_model.pth")

[1,  2000] loss: 2.257
[1,  4000] loss: 2.026
[1,  6000] loss: 1.910
[1,  8000] loss: 1.810
[1, 10000] loss: 1.693
[1, 12000] loss: 1.601
[2,  2000] loss: 1.551
[2,  4000] loss: 1.506
[2,  6000] loss: 1.472
[2,  8000] loss: 1.437
[2, 10000] loss: 1.411
[2, 12000] loss: 1.410
Finished Training


## Testing model

In [7]:
correct = 0
total = 0
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Files already downloaded and verified
Accuracy of the network on the 10000 test images: 32 %


## Square attack

In [ ]:

loss = 'margin_loss' #not targeted attack
dataset = 'cifar10'
eps = 12.75

import square_attack_v2.data
from square_attack_v2.data import load_cifar10
sys.path.append("/home/sandrech/code/ml/madry_cifar10/cifar10_data/cifar10_data")

x_test, y_test = load_cifar10(1000)

n_ex = 1000
n_iter = 10000
p = 0.05
metrics_path = 'fixed_metrics_file.metrics'
loss = 'cross_entropy'
#x_test, y_test = data
x_test, y_test = x_test[:n_ex], y_test[:n_ex]
if (x_test.shape[0] == 0):
    print("emty")
    #continue
y_target_onehot = utils.dense_to_onehot(y_test, n_cls=10) 

logits_clean = net.predict(x_test)
corr_classified = logits_clean.argmax(1) == y_test

n_queries, x_adv = square_attack_linf(net, x_test, y_target_onehot, corr_classified, eps, n_iter,
                                    p, metrics_path, False, loss)   


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


/tmp/ipykernel_213334/906728868.py:58: RuntimeWarning: divide by zero encountered in log
  loss = -np.log(probs[y])


1: acc=33.60% acc_corr=87.73% avg#q_ae=1.17 med#q=1.0, avg_margin=315.39 (n_ex=383, eps=12.750, 0.09s)
<class 'numpy.ndarray'>
2: acc=32.80% acc_corr=85.64% avg#q_ae=1.44 med#q=1.0, avg_margin=314.03 (n_ex=383, eps=12.750, 0.14s)
<class 'numpy.ndarray'>
3: acc=32.40% acc_corr=84.60% avg#q_ae=1.61 med#q=1.0, avg_margin=313.22 (n_ex=383, eps=12.750, 0.19s)
<class 'numpy.ndarray'>
4: acc=31.80% acc_corr=83.03% avg#q_ae=1.92 med#q=1.0, avg_margin=312.50 (n_ex=383, eps=12.750, 0.24s)
<class 'numpy.ndarray'>
5: acc=31.60% acc_corr=82.51% avg#q_ae=2.04 med#q=1.0, avg_margin=312.27 (n_ex=383, eps=12.750, 0.29s)
<class 'numpy.ndarray'>
6: acc=31.50% acc_corr=82.25% avg#q_ae=2.12 med#q=1.0, avg_margin=310.84 (n_ex=383, eps=12.750, 0.36s)
<class 'numpy.ndarray'>
7: acc=31.30% acc_corr=81.72% avg#q_ae=2.29 med#q=1.0, avg_margin=310.07 (n_ex=383, eps=12.750, 0.41s)
<class 'numpy.ndarray'>
8: acc=31.00% acc_corr=80.94% avg#q_ae=2.56 med#q=1.0, avg_margin=309.89 (n_ex=383, eps=12.750, 0.46s)
<class '

In [ ]:
print("DRRRRRRRRRR SKIBIDIDOP DOP DOP DOP YES YES YES YES SKIB SKIIBIDIDI SKBIDIDODOUBLEDOUBLE YES YES YES YES")